# Spaceship KNN Submission

Hello there whoever handles this aku lupa, ini cuma coba-coba aja si pake KNeighborsClassifier buat test submission. Feel free to modify ya, lov u makasi semangat. -Egar

In [ ]:
!pip install lightgbm==4.3.0


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



# Bring the Data In

In [ ]:
# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Model Training

In [ ]:

# Membuat pipeline

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LGBMClassifier(random_state=42))
])



param_grid = {
    'classifier__n_estimators': [500],
    'classifier__max_depth': [10],
    'classifier__learning_rate': [0.01],
    'classifier__subsample': [1],
    'classifier__colsample_bytree': [0.8]
}

# Menggunakan StratifiedKFold untuk handling imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


In [ ]:
# GridSearchCV setup
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='roc_auc', verbose=1)
grid_search.fit(X_train, y_train)

# Evaluate the model
best_model = grid_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
print("Best parameters:", grid_search.best_params_)
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 25170, number of negative: 93654
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004067 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1889
[LightGBM] [Info] Number of data points in the train set: 118824, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211826 -> initscore=-1.313954
[LightGBM] [Info] Start training from score -1.313954
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_le

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Inisialisasi RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline, 
    param_distributions=param_grid, 
    n_iter=50,  # Jumlah iterasi pencarian acak
    cv=cv, 
    scoring='roc_auc', 
    random_state=42,
    verbose=1,
    n_jobs=-1
)

# Melatih model
random_search.fit(X_train, y_train)

# Menampilkan hasil terbaik
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 1 is smaller than n_iter=50. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 25170, number of negative: 93654
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1889
[LightGBM] [Info] Number of data points in the train set: 118824, number of used features: 17
[Light

In [ ]:
# Evaluation on the test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
grid_search.fit(X_df, y_df)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
              precision    recall  f1-score   support

           0       0.89      0.95      0.92     26052
           1       0.75      0.58      0.66      6955

    accuracy                           0.87     33007
   macro avg       0.82      0.76      0.79     33007
weighted avg       0.86      0.87      0.87     33007

Confusion Matrix:
[[24737  1315]
 [ 2923  4032]]
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 31429, number of negative: 117101
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005966 second

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        LGBMClassifier(random_state=42))]),
             param_grid={'classifier__colsample_bytree': [0.8],
                         'classifier__learning_rate': [0.01],
                         'classifier__max_depth': [10],
                         'classifier__n_estimators': [500],
                         'classifier__subsample': [1]},
             scoring='roc_auc', verbose=1)

In [ ]:
# Melakukan cross-validation dengan GridSearchCV
scores = cross_val_score(grid_search, X_df, y_df, cv=5)

# Menampilkan hasil cross-validation
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 25143, number of negative: 93681
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017343 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1889
[LightGBM] [Info] Number of data points in the train set: 118824, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211599 -> initscore=-1.315316
[LightGBM] [Info] Start training from score -1.315316
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you did

In [ ]:
# Evaluation on the test set
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
grid_search.fit(X_df, y_df)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
              precision    recall  f1-score   support

           0       0.90      0.95      0.92     26052
           1       0.76      0.59      0.66      6955

    accuracy                           0.87     33007
   macro avg       0.83      0.77      0.79     33007
weighted avg       0.87      0.87      0.87     33007

Confusion Matrix:
[[24765  1287]
 [ 2877  4078]]
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 31429, number of negative: 117101
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005561 second

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        LGBMClassifier(random_state=42))]),
             param_grid={'classifier__colsample_bytree': [0.8],
                         'classifier__learning_rate': [0.01],
                         'classifier__max_depth': [10],
                         'classifier__n_estimators': [500],
                         'classifier__subsample': [1]},
             scoring='roc_auc', verbose=1)

In [ ]:
# Predict probabilities for submission data
y_submission_proba = grid_search.predict_proba(X_submission)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[0.01788578 0.84044794 0.02779601 ... 0.028606   0.183122   0.16679128]


,id,Exited
0,165034,0.017886
1,165035,0.840448
2,165036,0.027796
3,165037,0.232029
4,165038,0.383344
...,...,...
110018,275052,0.043406
110019,275053,0.101982
110020,275054,0.028606
110021,275055,0.183122


In [ ]:
submission_dict.to_csv('bankChurn_light3.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>